In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Qwen2ForCausalLM, pipeline
from torch.utils.data import Dataset


In [2]:

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", output_hidden_states=True, return_dict_in_generate=True)

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [38]:
# Create a pipeline for text generation
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)  # Use device=0 for the first GPU

# Example usage
messages = [
    {"role": "user", "content": "Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q."},
]
response = pipe(messages)
print(response)

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.'}, {'role': 'assistant', 'content': 'Okay, so I need to find the degree of the field extension Q(sqrt(2), sqrt('}]}]


In [34]:
text = "Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q."
inputs = tokenizer(text, return_tensors="pt")
print(inputs["input_ids"].shape)
outputs = model.generate(**inputs, max_length=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
outputs

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


torch.Size([1, 26])


GenerateDecoderOnlyOutput(sequences=tensor([[151646,   9885,    279,   8381,    369,    279,   2661,   2070,   8894,
           1207,  83073,      7,     17,    701,  17946,      7,     18,    701,
          17946,      7,     16,     23,    593,    916,   1207,     13,    358,
            646,    944,   7071,    700,   1246,    311,   5486,    419,     13,
           5209,   1492,     13,    358,   1184,    311,   7071,    700,    279,
           8381,    369,    279,   2661,   2070,   8894,   1207,  83073,      7,
             17,    701,  17946,      7,     18,    701,  17946,      7,     16,
             23,    593,    916,   1207,     13,    358,    646,    944,   7071,
            700,   1246,    311,   5486,    419,     13,   5209,   1492,     13,
            358,   1184,    311,   7071,    700,    279,   8381,    369,    279,
           2661,   2070,   8894,   1207,  83073,      7,     17,    701,  17946,
              7]]), scores=None, logits=None, attentions=None, hidden_sta

In [35]:
predicted_token_ids = torch.argmax(outputs["logits"], dim=-1)
decoded_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True) #decodes the first batch.

In [35]:
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
generated_text

"Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. I can't figure out how to approach this. Please help. I need to figure out the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. I can't figure out how to approach this. Please help. I need to figure out the degree for the given field extension Q(sqrt(2), sqrt("

In [70]:
len(outputs.sequences[0])

100

In [71]:
len(outputs.hidden_states), len(outputs.hidden_states[0])

(74, 29)

In [ ]:
# concatenate hidden states

for i in range(1, len(outputs.hidden_states)):
    for j in range(len(outputs.hidden_states[i])):
        print(outputs.hidden_states[0][j].shape, outputs.hidden_states[i][j].shape)
        # print(i,j)

torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])
torch.Size([1, 1, 1536])


In [ ]:
outputs.hidden_states[0][28].shape  # output.hidden_states[]

torch.Size([1, 26, 1536])

In [58]:
torch.save(outputs.hidden_states, "data/hidden_state_0.pt")

In [30]:
outputs["logits"].shape

torch.Size([1, 26, 151936])

In [15]:
outputs.hidden_states[0][0].shape  # batch, sequence ind

torch.Size([1, 26, 1536])

In [24]:
outputs.hidden_states[70][10]

tensor([[[ 4.0748,  0.8844, -0.0042,  ..., -0.3326, -0.4221, -0.2327]]])

In [29]:
outputs["logits"][0]

tensor([[ 3.7575,  3.1718,  4.4061,  ..., -5.3510, -5.3516, -5.3517],
        [ 4.6869,  3.5484, -0.8713,  ...,  0.9761,  0.9759,  0.9728],
        [ 5.4803,  2.8965,  1.8519,  ...,  1.8977,  1.8979,  1.8942],
        ...,
        [ 6.0502,  3.8007,  0.5189,  ..., -0.9137, -0.9147, -0.9169],
        [ 7.4730,  9.0720,  5.3566,  ..., -1.1273, -1.1275, -1.1289],
        [ 2.4215,  0.5701,  2.1570,  ..., -1.3473, -1.3476, -1.3480]],
       grad_fn=<SelectBackward0>)

In [60]:
pipe.model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [16]:
outputs.hidden

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'hidden'

In [56]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label